In [ ]:
# 종속 패키지(dependencies) 설치(pytorch, detectron2)
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [1]:
# 인스톨이 완료되면 Colab Runtime을 다시 시작해주세요. 

# 기본 설정
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools

# detectron2 logger 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 자주 사용하는 라이브러리 임폴트
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# 자주 사용하는 detectron2 유틸 임폴트 
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog

In [2]:
!git clone https://github.com/sosobam/AI_Jump_up.git

Cloning into 'AI_Jump_up'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 88 (delta 6), reused 11 (delta 3), pack-reused 65
Unpacking objects: 100% (88/88), done.
Checking out files: 100% (61/61), done.


In [ ]:
# detectron2에서 데이터셋 정보를 로드하는 형식에 맞춰 함수를 작성합니다.
def get_PTN_dicts(img_dir):
    json_file = os.path.join(img_dir, "annotations.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    # 최종 결과물
    dataset_dicts = []
    
    idx = 0
    # 이미지들을 annotation한 json에서 이미지별로 하나씩 데이터 뽑아서 record에 저장(key : 이미지file.jpg, val : seg데이터)
    for key, val in imgs_anns.items():
        record = {} # 각 이미지별 정보 담을 dataset 포맷
        
        filename = os.path.join(img_dir, key) # 이미지file 경로
        height, width = cv2.imread(filename).shape[:2] # 이미지file 경로로 cv2로 이미지 열어서 height, width 추출
        
        # 이미지 기본정보
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = val["instances"]
        objs = []

        # 이미지 기본정보 중 annotation_class별로 point들(x,y,x,y,x,y,...x,y순서) 뽑아서 obj에 담고 objs에 class별로 누적 후 record["annotations"]에 최종 저장
        for anno in annos:
            points = anno["points"]
            px = []
            py = []

            # point들(x,y,x,y,x,y,...x,y순서) 뽑아서 px, py로 각각 list 생성
            for i in range(0,int(len(points)/2)):
              px = px + [points[i*2]] # 짝수
              py = px + [points[i*2+1]] # 홀수

            # px, py list를 poly에 (x,y)형태로 묶어서 다시 list화
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = list(itertools.chain.from_iterable(poly))

            # obj dict에 (x,y)list를 토대로 Box만들고 list 등록
            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
                "iscrowd": 0
            }
            objs.append(obj) # obj를 objs에 누적
        record["annotations"] = objs # objs를 record["annotations"]에 등록
        dataset_dicts.append(record) # record를 dataset에 누적
    return dataset_dicts

In [ ]:
# train, val 이미지들을 metadata set에 등록
for d in ["train", "val"]:
    DatasetCatalog.register("PTN_" + d, lambda d=d: get_PTN_dicts(d))
    MetadataCatalog.get("PTN_" + d).set(thing_classes=["PTN"])
balloon_metadata = MetadataCatalog.get("PTN_train")

In [ ]:
# 데이터가 제대로 로드되었는지 확인
dataset_dicts = get_PTN_dicts("AI_Jump_up/TRAIN")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])